In [14]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import glob
import multiprocessing

from operator import attrgetter
from sklearn.datasets import make_biclusters
from sklearn.datasets import samples_generator as sg
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN

from metrics import *

In [15]:
data = pd.read_csv('TestData/SimulatedDataCoherence/BiclusterA_high.csv',header=None)
scaler = MinMaxScaler()
scaler.fit(data)
# data = scaler.transform(data)*2-1

data = np.concatenate((data,data),axis=0)

In [16]:
class Bicluster(object):
    
    def __init__(self, data, rho=None, gamma=None, H = -1, HT = -1):
        """
        Bicluters structure
        
        :param rho: <np.array[int]> the map of the indices of rows of the bicluster i.e [2,4,7,8]
        :param gamma: <np.array[int]> the map of gammas, same structure f rho
        :param H: <float> the H values of the bicluster
        """
        self.gamma = gamma
        self.rho = rho
        self.H = H
        self.HT = HT
        self._mutation = 0
        self.data = data[np.ix_(self.rho, self.gamma)]
        
    @property
    def mutation(self):
        return self._mutation
    
    def __str__(self):
        return ("rho: \n"+ str(self.rho) + "\ngamma: \n" + str(self.gamma) + "\nH: \n " + str(self.H))
        
    def update_data(self,rho = None, gamma = None):
        self.data = data[np.ix_(self.rho, self.gamma)]
        self.compute_H()
        
    def row_length(self):
        return len(self.rho)
    
    def col_length(self):
        return len(self.gamma)
    
    def compute_H(self):
        """
        Method:
        Computes the coherence H of the data
        """
        if (self.H<0):
            self.H = PairBasedCoherence(self.data).HP
        return self.H

In [65]:
class MapArray(object):
    def __init__(self, map_array = None, flg_direction = None):
        """
        Array that will map the indices whether its a row map or a col map
        """
        self._map_array = map_array
        self._flg_direction = flg_direction
        
    @property
    def map_array(self):
        return self._map_array
    
    @property
    def flg_direction(self):
        return self._flg_direction


In [66]:
def split_cols(data):
    """
    Function:
    Spits the bicluster according to the best split in of columns
    
    :param data: <np.array[np.array]> data of the bicluster to split
    :param bicluster: <Bicluster> data 
    """
    map_array = MapArray(np.zeros(data.shape[1]),'cols')
    middle = 0
    max_h = 0
    for i in range(1,data.shape[1]):
        # print(" i " + str(i))
        temp_h_right = PairBasedCoherence(data[:,i:]).HP
        # print("right " + str(temp_h_right))
        temp_h_left = PairBasedCoherence(data[:,:i]).HP
        # print("left " + str(temp_h_left))
        temp_max = max(temp_h_right, temp_h_left)
        
        if (temp_max >= max_h):
            max_h = temp_max
            middle = i
            
    print(middle)
    map_array.map_array[0:middle] = np.ones(middle)
    print("Final map " + str(map_array.map_array))
    return map_array, max_h

In [67]:
def split_rows(data):
    """
    Apply DBSCAN over the bicluster to filter noise
    
    :param bicluster: <Bicluster> data
    """
    n_cols = data.shape[1]
    dm = np.asarray([[((PositiveNegativeCorrelation(p1,p2,len(p1)).H_pos) 
                       if (PositiveNegativeCorrelation(p1,p2,len(p1)).H_pos)!= 1 else 0.0) for p2 in data] for p1 in data])
    
    db = DBSCAN(eps=0.01,metric='precomputed', min_samples=5
           ).fit(1-dm**4)
    print(db.labels_)
    map_array = MapArray(db.labels_,'cols')
    return map_array, max_h

In [68]:
def compute_bicluster_coherence(bicluster):
    """
    Splits bicluster and chooses wether its better to split rows or columns
    
    :param bicluster: <Bicluster> Bicluster to split
    """
    map_array_cols, h_cols = split_cols(bicluster.data)
    map_array_rows, h_rows = split_rows(bicluster.data)
    
    if (h_cols > h_rows):
        return map_array_cols, h_cols, "cols"
    else:
        return map_array_rows, h_rows, "rows"

In [79]:
class STSSCAN(object):
    
    def __init__(self, data, n_clusters = 2):
        """
        STSSCAN
        
        Applies biclustering over data
        
        :param data: <np.array> The data to apply biclusters
        :param n_clusters: <int> The number of biclusters to find
        
        """
        assert data is not None, "Empty data"
        self._data = data
        """ Data to apply biclustering """
        self._I, self._J = self._data.shape
        """ Stores shape of data """
        self._n_clusters = n_clusters
        """ Number of clusters to find """
        self._biclusters = list()
        """ List of biclusters composed by Bicluster Class"""
        self._objective_function = 0
        """ Objective function: maximize with boundary 1"""
    
    @property
    def n_clusters(self):
        return self._n_clusters
    
    @property
    def data(self):
        return self._data
    
    @property
    def labels(self):
        return _labels
            
    def fit(self):
        """
        Fits the data on the algorithm. Iters over the 
        number of biclusters until find the requested number
        """
        n_iterations = 0
        n_clusters = 1
        initial_bicluster = Bicluster(self._data,np.arange(self._I), np.arange(self._J))
        self._biclusters.append(initial_bicluster)
        print("fit")
        for n_iterations in range(0,n_clusters):
            self._split_biclusters()
                
    def _split_biclusters(self):
        
        """
        Splits bicluters in order to find the next biclusters that optimizes
        the sum of coherences among biclusters
        """
        """ Initialize variables """
        import ipdb; ipdb.set_trace()
        print(" splitting biclusters ")
        max_delta = 0
        tmp_bicluster_delta = 0
        tmp_bisection = MapArray()
        tmp_split = None
        split = None
        position = 0
        best_position = 0
        best_bisection = MapArray()
        """ Iterate over the biclusters"""
        for bicluster in self._biclusters:
            """ If shape of the biclusters is within the bounds """ 
            if (bicluster.row_length() >= (np.log(self._I)*5) and bicluster.col_length() >= 2):
                
                """ Compute the bicluster coherence """
                tmp_bisection, tmp_bicluster_delta = compute_bicluster_coherence(bicluster)
                print(tmp_bisection)
                """ Store the best split of biclusters """
                if(tmp_bicluster_delta >= max_delta):
                    split = tmp_split
                    max_delta = tmp_bicluster_delta
                    best_bicluster = bicluster
                    best_position = position
                    best_bisection = tmp_bisection
            position += 1
        """ Add the bicluster to the list of biclusters """
        self._add_bicluster(best_bicluster, best_position, best_bisection)
    
    def _add_bicluster(self, best_bicluster, best_position, bisection):
        """
        Adds the found bicluster into the list of labels
        """
        pass 
        """ If the bisection is on rows """
        if bisection.flg_direction == 'row':
            
            """Create and append the new bicluster spliting rows """
            self._labels[best_position] = Bicluster(self._data, best_bicluster.rho[np.flatnonzero(bisection.map_array == 0)]
                                                    ,best_bicluster.gamma)
            self._labels.append(Bicluster(self._data, best_bicluster.rho[np.flatnonzero(bisection.map_array == 1)]
                                                    ,best_bicluster.gamma))
        else:
            """Create and append the new bicluster spliting columns """
            self._labels[best_position] = Bicluster(self._data, best_bicluster.rho
                                                    ,best_bicluster.gamma[np.flatnonzero(bisection.map_array == 0)])
            self._labels.append(Bicluster(self._data, best_bicluster.rho
                                                    ,best_bicluster.gamma[np.flatnonzero(bisection.map_array == 1)]))

In [80]:
stsscan = STSSCAN(np.array(data),5)

In [ ]:
stsscan.fit()